## Clasificarea imaginilor folosind descriptori

In [5]:
import glob

import cv2 as cv
import numpy as np
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from joblib import dump, load
from skimage.feature import hog
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from torch.autograd import Variable
from copy import deepcopy

### Incarcarea imaginilor de antrenare

In [6]:
path = '..\\resources\\train\\'
folders = glob.glob(path + '*')
print(folders)

['..\\resources\\train\\piese_portocalii', '..\\resources\\train\\piese_verzi', '..\\resources\\train\\pozitii_libere']


In [7]:
train_images = []
train_labels = []

In [8]:
folders = [
    "piese_portocalii",
    "piese_verzi",
    "pozitii_libere"
]
for clas in folders:
    # clas=f.replace(path,'')
    print(clas)
    files = glob.glob(path + clas + '\\*jpg')
    for i in files:
        image = cv.imread(i)
        image = cv.resize(image, (50, 50), interpolation=cv.INTER_LINEAR)
        img = np.asarray(image)
        train_images.append(img)
        train_labels.append(clas)
train_images = np.array(train_images)
train_labels = np.array(train_labels)

piese_portocalii
piese_verzi
pozitii_libere


In [9]:
print(train_images.shape)
print(train_labels.shape)

(772, 50, 50, 3)
(772,)


### Incarcarea imaginilor de testare

In [10]:
path = '..\\resources\\test\\'

folders = glob.glob(path + '*')
print(folders)

['..\\resources\\test\\piese_portocalii', '..\\resources\\test\\piese_verzi', '..\\resources\\test\\pozitii_libere']


In [11]:
test_images = []
test_labels = []

In [12]:
folders = [
    "piese_portocalii",
    "piese_verzi",
    "pozitii_libere"
]
for clas in folders:
    print(clas)
    files = glob.glob(path + clas + '\\*jpg')
    for i in files:
        image = cv.imread(i)
        image = cv.resize(image, (50, 50), interpolation=cv.INTER_LINEAR)
        img = np.asarray(image)
        test_images.append(img)
        test_labels.append(clas)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

piese_portocalii
piese_verzi
pozitii_libere


In [13]:
print(test_images.shape)
print(test_labels.shape)

(179, 50, 50, 3)
(179,)


### Diferite tipuri de descriptori

In [14]:
model = models.resnet18(pretrained=True)
layer = model._modules.get('avgpool')
model.eval()
# print(model)

C:\Users\allex\AppData\Local\Programs\Python\Python38\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\allex\AppData\Local\Programs\Python\Python38\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [15]:
def get_resnet_descriptors(path):
    images = []
    labels = []
    folders = [
        "piese_portocalii",
        "piese_verzi",
        "pozitii_libere"
    ]
    for clasa in folders:
        # clas=f.replace(path,'')

        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225])
        to_tensor = transforms.ToTensor()
        scaler = transforms.Resize((50, 50))
        model = models.resnet18(pretrained=True)
        layer = model._modules.get('avgpool')
        model.eval()


        print(clasa + " " + path)
        files = glob.glob(path + clasa + '\\*jpg')
        for file in files:
            img_cv = cv.imread(file)

            img_PIL = Image.fromarray(img_cv)
            t_img = Variable(normalize(to_tensor(scaler(img_PIL))).unsqueeze(0))
            my_embedding = torch.zeros(512)
            def copy_data(m, i, o):
                my_embedding.copy_(o.data.reshape(o.data.size(1)))
            h = layer.register_forward_hook(copy_data)
            model(t_img)
            h.remove()
            my_embedding = my_embedding.numpy()

            images.append(my_embedding)
            labels.append(clasa)
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

In [16]:
def get_flatten_images(images):
    images_flatten = []
    for image in images:
        image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
        image = image.flatten()
        images_flatten.append(image)
    images_flatten = np.array(images_flatten)
    return images_flatten

In [17]:
def get_gray_histogram(images):
    gray_histograms = []
    for image in images:
        image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
        hist = cv.calcHist([image], [0], None, [256], [0, 256])
        gray_histograms.append(hist.flatten())
    gray_histograms = np.array(gray_histograms)
    return gray_histograms

In [18]:
def get_rgb_histogram(images):
    color_histograms = []
    for img in images:
        hist = cv.calcHist([img], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
        color_histograms.append(hist.flatten())
    color_histograms = np.array(color_histograms)
    return color_histograms

In [19]:
def get_hog_descriptors(images):
    descriptors = []
    for img in images:
        img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        features = hog(img, pixels_per_cell=(8, 8), orientations=9, cells_per_block=(2, 2), block_norm='L2-Hys', feature_vector=True)
        descriptors.append(features)
    descriptors = np.array(descriptors)
    return descriptors

### Calcularea descriptorilor pentru imaginile de antrenare si testare

In [20]:
train_images_flatten = get_flatten_images(train_images)
test_images_flatten = get_flatten_images(test_images)

In [21]:
train_images_gray_hist = get_gray_histogram(train_images)
test_images_gray_hist = get_gray_histogram(test_images)

In [22]:
train_images_color_hist = get_rgb_histogram(train_images)
test_images_color_hist = get_rgb_histogram(test_images)

In [23]:
train_images_hog = get_hog_descriptors(train_images)
test_images_hog = get_hog_descriptors(test_images)

In [24]:
path = '..\\resources\\train\\'
train_images_resnet, train_labels_resnet = get_resnet_descriptors(path)

path = '..\\resources\\test\\'
test_images_resnet, test_labels_resnet = get_resnet_descriptors(path)

piese_portocalii ..\resources\train\
piese_verzi ..\resources\train\
pozitii_libere ..\resources\train\
piese_portocalii ..\resources\test\
piese_verzi ..\resources\test\
pozitii_libere ..\resources\test\


### Antrenarea unui SVM liniar cu diferiti descriptori

In [25]:
print(train_labels[:3])

['piese_portocalii' 'piese_portocalii' 'piese_portocalii']


In [26]:
best_accuracy = 0
best_c = 0
best_model = None
Cs = [10 ** -5, 10 ** -4, 10 ** -3, 10 ** -2, 10 ** -1, 10 ** 0]
for c in Cs:
    print('Antrenam un clasificator pentru c=%f' % c)
    model = LinearSVC(C=c)
    model.fit(train_images_resnet, train_labels_resnet)
    acc = model.score(train_images_resnet, train_labels_resnet)
    print(acc)
    if acc > best_accuracy:
        best_accuracy = acc
        best_c = c
        best_model = deepcopy(model)

print('Performanta clasificatorului optim pt c = %f' % best_c)

# salvam cel mai bun model
dump(best_model,"model_clasificare_piesa.joblib")

# incarcam modelul
load("model_clasificare_piesa.joblib")

Antrenam un clasificator pentru c=0.000010
0.9974093264248705
Antrenam un clasificator pentru c=0.000100
1.0
Antrenam un clasificator pentru c=0.001000
1.0
Antrenam un clasificator pentru c=0.010000
1.0
Antrenam un clasificator pentru c=0.100000
1.0
Antrenam un clasificator pentru c=1.000000
1.0
Performanta clasificatorului optim pt c = 0.000100


C:\Users\allex\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(C=0.0001)

In [27]:
predicted_labels = best_model.predict(test_images_resnet)
# print(predicted_labels)
print(len(predicted_labels))
print(accuracy_score(test_labels_resnet, predicted_labels))

179
1.0
